In [ ]:
import sys
import warnings
from newspaper import Article
from tqdm import tqdm
from textblob import TextBlob

from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.corpus import stopwords


warnings.filterwarnings("ignore")

# Import main utility functions
sys.path.insert(0, r'c:\Users\joneh\master_thesis\src')
from main_utils import *
from db_utils import *

In [ ]:
news_df = news_db_load('news')


In [ ]:
text_dict = {}

i = 0
for index, row in tqdm(news_df.iterrows(), nrows=len(news_df)):
    url = row['web_url']
    id = row['article_id']
    article = Article(url)

    # print article text
    article.download()
    article.parse()
    
    text_dict[id] = article.text

    i += 1

    if i > 2000:
        break


In [ ]:
text_df = pd.DataFrame.from_dict(text_dict, orient='index', columns=['text'])

# use textblob to get sentiment
text_df[['polarity', 'subjectivity']] = text_df['text'].apply(lambda x: pd.Series(TextBlob(x).sentiment))

# use nltk to get sentiment
sia = SentimentIntensityAnalyzer()
text_df['SIA_sentiment'] = text_df['text'].apply(lambda x: sia.polarity_scores(x)['compound'])

In [ ]:

fig = text_df['polarity'].hist(bins=100)

# set x-lim to -1 and 1
fig.set_xlim(-1, 1)

In [ ]:
# get the 5 most negative articles
print(text_df.sort_values(by='polarity', ascending=False).head(5)['text'].values)